In [1]:
#%matplotlib inline
import OifFile as of
import numpy as np
import matplotlib.pyplot as plt
import os
from __future__ import division
import sys
sys.path.insert(1,'T:\\James_Rowland\\alpha6_Cre_ERT')

def file_loader():
    '''Loads Tiff images and returns them as arrays in a list''' 
    #get list of all files in dir
    dirList = os.listdir('T:\\James_Rowland\\alpha6_Cre_ERT\\150911')   
    
    dirList.sort(key=lambda x: os.stat(os.path.join('T:\\James_Rowland\\alpha6_Cre_ERT\\150911', x)).st_mtime)
    
    return dirList
    
def oif_selector():  

    oif_files = []
    
    file_list = file_loader()
    
    for files in file_list:
        if files.endswith((".oif")):
            oif_files.append(files)
    
    #print date_sorted_files
   
    return oif_files

def array_list_creator():
        
    oif_file_list = oif_selector()

    
    #load the files corresponding to the lists into lists of arrays
    oif_array_list = []
    
    for fileName in oif_file_list:
        oif_array_list.append(of.OifFile(fileName).asarray())
    
    #finally return the list of arrays
    return oif_array_list

def get_percentage_expressing():
    
    arraylist = array_list_creator()
    
    y = [] 
    
    for item in arraylist:
       
        DAPI = item[:,0,:,:]
        TdTomato = item[:,1,:,:]
        #indexes to split the two channels

        D_reshaped = DAPI.reshape(DAPI.shape[0], -1)
        Td_reshaped = TdTomato.reshape(TdTomato.shape[0], -1)
        # reshapes the array to 39L, 262144L - merging 512 x 512 into a single dimension

        Dav = np.average(D_reshaped, axis=1)
        Tdav = np.average(Td_reshaped, axis=1)
        # average pixel intensity of each slice 

        Dstd = np.std(D_reshaped, axis = 1)
        Tdstd = np.std(Td_reshaped, axis = 1)
        # standard deviation of pixel intensities of each slice 

        DAPI_thresh = Dav + Dstd
        #this threshold will only hold true at bright nuclei

        DAPIbroad = DAPI_thresh[:, np.newaxis, np.newaxis]
        # generates new dimensions to the DAPI threshold, allowing it to be used in operations

        import numpy.ma as ma

        masked_DAPI = ma.masked_greater(DAPI, DAPIbroad)
        #returns true if the value definitely lies in a nucleus

        Td_background = masked_DAPI.mask * TdTomato
        Td_back_reshape = Td_background.reshape(TdTomato.shape[0], -1)
        back_av = np.average(Td_back_reshape, axis = 1, weights=Td_back_reshape.astype(bool))
        #generates the average intensity of the background at each slice - weight excludes 0 values 

        Td_thresh = back_av + Tdstd
        Tdbroad = Td_thresh[:, np.newaxis, np.newaxis]
        masked_Td = ma.masked_greater(TdTomato, Tdbroad)
        # generates a masked array returning true for pixels over threshold

        number_pixels_above_threshold = np.sum(masked_Td.mask)
        number_background_pixels = np.sum(masked_DAPI.mask)

        percentage_expressing = (number_pixels_above_threshold / ((DAPI.shape[0]*DAPI.shape[1]*DAPI.shape[2])-number_background_pixels))*100

        y.append(percentage_expressing)
        
    results = y
    
    print results
    
    return results


def  graph_plotter():    
    results_to_plot = get_percentage_expressing()

    number_of_bars = range(len(results_to_plot))

    x_axis = file_loader()
    chart = plt.bar(number_of_bars, results_to_plot, align='center')
    x_labels = plt.xticks(number_of_bars, x_axis, rotation=90)
    y_title = plt.ylabel('Percentage Pixels Above Threshold')
    x_title = plt.xlabel('Animal')

    plt.show()
    
graph_plotter()















[4.3938769689094537, 5.5618403230203519, 2.6995129913295686, 3.1566616658788718, 2.7387196388483583, 2.107637363531345, 3.0864877332164178]


C:\Users\zcbtjro\Downloads\Python2.7\python-2.7.9.amd64\lib\site-packages\tifffile.py:158: UserWarning: failed to import the optional _tifffile C extension module.
Loading of some compressed images will be slow.
Tifffile.c can be obtained at http://www.lfd.uci.edu/~gohlke/
  "failed to import the optional _tifffile C extension module.\n"


In [59]:
#this tests the code using a two 5 x 5 x 5 arrays


%matplotlib inline
from tifffile import TiffFile as tf
import numpy as np
import matplotlib.pyplot as plt

DAPI_test = np.arange(125).reshape((5, 5, 5))
TdTomato_test = np.arange(125).reshape((5, 5, 5))

D_reshaped = DAPI_test.reshape(DAPI_test.shape[0], -1)
Td_reshaped = TdTomato_test.reshape(TdTomato_test.shape[0], -1)
# reshapes the array to 39L, 262144L - merging 512 x 512 into a single dimension

Dav = np.average(D_reshaped, axis=1)
Tdav = np.average(Td_reshaped, axis=1)
# average pixel intensity of each slice 

Dstd = np.std(D_reshaped, axis = 1)
Tdstd = np.std(Td_reshaped, axis = 1)
# standard deviation of pixel intensities of each slice 

DAPI_thresh = Dav  + Dstd 
#this threshold will only hold true at bright nuclei

DAPIbroad = DAPI_thresh[:, np.newaxis, np.newaxis]
# generates new dimensions to the DAPI threshold, allowing it to be used in operations

import numpy.ma as ma

masked_DAPI = ma.masked_greater(DAPI_test, DAPIbroad)
#returns true if the value definitely lies in a nucleus

Td_background = masked_DAPI.mask * TdTomato_test
Td_back_reshape = Td_background.reshape(TdTomato_test.shape[0], -1)
back_av = np.average(Td_back_reshape, axis = 1, weights=Td_back_reshape.astype(bool))

#generates the average intensity of the background at each slice 

Td_thresh = back_av + Tdstd
Tdbroad = Td_thresh[:, np.newaxis, np.newaxis]
masked_Td = ma.masked_greater(TdTomato_test, Tdbroad)
# generates a masked array returning true for pixels over threshold

number_pixels_above_threshold = np.sum(masked_Td.mask)
number_background_pixels = np.sum(masked_DAPI.mask)

from __future__ import division

percentage_expressing = (number_pixels_above_threshold / ((DAPI_test.shape[0]*DAPI_test.shape[1]*DAPI_test.shape[2])-number_background_pixels))*100

print Td_background
print back_av
back_av.shape



[[[  0   0   0   0   0]
  [  0   0   0   0   0]
  [  0   0   0   0   0]
  [  0   0   0   0   0]
  [ 20  21  22  23  24]]

 [[  0   0   0   0   0]
  [  0   0   0   0   0]
  [  0   0   0   0   0]
  [  0   0   0   0   0]
  [ 45  46  47  48  49]]

 [[  0   0   0   0   0]
  [  0   0   0   0   0]
  [  0   0   0   0   0]
  [  0   0   0   0   0]
  [ 70  71  72  73  74]]

 [[  0   0   0   0   0]
  [  0   0   0   0   0]
  [  0   0   0   0   0]
  [  0   0   0   0   0]
  [ 95  96  97  98  99]]

 [[  0   0   0   0   0]
  [  0   0   0   0   0]
  [  0   0   0   0   0]
  [  0   0   0   0   0]
  [120 121 122 123 124]]]
[  22.   47.   72.   97.  122.]


(5L,)

In [38]:
%matplotlib inline
import OifFile as of
import numpy as np
import matplotlib.pyplot as plt
import os
from __future__ import division
import sys
sys.path.insert(1,'T:\\James_Rowland\\alpha6_Cre_ERT')

def file_loader():
    '''Loads Tiff images and returns them as arrays in a list''' 
    #get list of all files in dir
    dirList = os.listdir('T:\\James_Rowland\\alpha6_Cre_ERT\\150617')   
    
    oif_files = []
    
    for files in dirList:
        if files.endswith((".oif")):
            oif_files.append(files)
    
    date_sorted_files = oif_files.sort(key=lambda x: os.stat(os.path.join('T:\\James_Rowland\\alpha6_Cre_ERT\\150617', x)).st_mtime)
    
    print date_sorted_files
   
    return date_sorted_files


#def MyFn(s):
    #print s[-1]
    #return s[-1]
      
#hi = file_loader()
    
#ol = sorted(hi, key=MyFn)

#print ol

file_loader()


NameError: global name 'oif_fgyjiles' is not defined

In [30]:
import gdal_merge

ImportError: No module named gdal_merge